# A (Possible) Code for Calibrating the Tagger Timing

First, we import the tools we'll need:

In [1]:
import Acqu as aq
import AcquDetector as aqdet
import ROOT
import numpy as np
from rootpy.plotting import histogram, Hist2D, Hist, Canvas

Welcome to JupyROOT 6.16/00


Import the data to be analyzed:

In [2]:
inFile = '/scratch/2019-05_Timepix/Timepix_33.dat'
aq.openFile(inFile)

Mk2 Data


Load in the detector file for the Tagger:

In [3]:
aqdet.LoadDetectors(['taggerNew.json'])

taggerNew.json


Now we plot the data for the tagger channels and apply a gaussian fit (with an offset) to them (note that onl tbe first 10 channels are done, as it's very slow running to do all the channels at once):

In [7]:
taggerChannels = aqdet.Channels['Tagger'] # number of channels in the tagger (368)

myFit = ROOT.TF1("myFit","gaus(0)+pol0(3)")
myFit.SetParameter(1,880)
myFit.SetParameter(2,10)
myFit.SetParNames("constant","mean","stdev","offset")


# histos = [None]*taggerChannels
histos = [None]*10

# for i in range(taggerChannels):
for i in range(10):
    histos[i] = Hist(1200,300,1500)

def plotCalTagger():
    data = aqdet.Calibrate(aq.adcArray)
    taggerTimes = aqdet.Arrays['Tagger']['Time']
    
    for dat in taggerTimes:
        for time in dat['Time']:
            # for i in range(taggerChannels):
            for i in range(10):
                if (dat['channel']==(i)):
                    histos[i].Fill(time)  
    
aq.runFunction(plotCalTagger,0,50000)

ROOT.enableJSVis()

# c1 = ROOT.TCanvas("my1","Title",1000,(taggerChannels/16)*1000)
# c1.Divide(4,taggerChannels/4,0,0)
c1 = ROOT.TCanvas("my1","Title",1000,2500)
c1.Divide(2,5,0,0)

print("almost done")

# for i in range(taggerChannels):
for i in range(10):
    c1.cd(i+1)
    histos[i].SetTitle("Channel "+str(i))
    histos[i].Draw("colz")
    histos[i].Fit('myFit')
    print(histos[i].GetParam(2))
c1.Draw()


almost done


AttributeError: 'Hist' object has no attribute 'GetParam'

 FCN=1242.24 FROM MIGRAD    STATUS=CONVERGED     254 CALLS         255 TOTAL
                     EDM=1.38585e-06    STRATEGY= 1      ERROR MATRIX ACCURATE 
  EXT PARAMETER                                   STEP         FIRST   
  NO.   NAME      VALUE            ERROR          SIZE      DERIVATIVE 
   1  constant     2.39392e+01   1.50416e+00   2.18878e-02  -6.99260e-04
   2  mean         8.80786e+02   5.85320e-01   9.99502e-03   2.07127e-03
   3  stdev        8.92479e+00   4.48927e-01   6.47290e-03  -3.78660e-04
   4  offset       7.28029e+00   8.65261e-02   1.47188e-03  -8.93744e-03


Setting them to the ground?:

Testing Zone:

In [43]:
taggerChannels = aqdet.Channels['Tagger'] # number of channels in the tagger (368)

bins = 1200
h1 = Hist(bins,300,1500)

def plotCalTagger():
    data = aqdet.Calibrate(aq.adcArray)
    taggerTimes = aqdet.Arrays['Tagger']['Time']
    
    for dat in taggerTimes:
        for time in dat['Time']:
            if (dat['channel']==1):
                h1.Fill(time)
        
    
aq.runFunction(plotCalTagger,0,50000)

ROOT.enableJSVis()

c1 = ROOT.TCanvas("my1","Title",1000,1000)

h1.SetTitle("Channel 1")
h1.GetXaxis().SetTitle("Time")
h1.Draw("colz")
myFit = ROOT.TF1("myFit","gaus(0)+pol0(3)")
myFit.SetParameter(1,880)
myFit.SetParameter(2,10)
h1.Fit('myFit')


con = h1.Fit('myFit', 'SQ').Get().Parameter(0)
mean = h1.Fit('myFit', 'SQ').Get().Parameter(1)
stdev = h1.Fit('myFit','SQ').Get().Parameter(2)
offset = h1.Fit('myFit','SQ').Get().Parameter(3)
print(con,mean,stdev,offset)

c1.Draw()

(18.782896336644132, 879.1420867213186, 8.825971735291892, 7.266982750767076)
 FCN=1256.07 FROM MIGRAD    STATUS=CONVERGED     266 CALLS         267 TOTAL
                     EDM=2.3061e-08    STRATEGY= 1      ERROR MATRIX ACCURATE 
  EXT PARAMETER                                   STEP         FIRST   
  NO.   NAME      VALUE            ERROR          SIZE      DERIVATIVE 
   1  p0           1.87829e+01   1.39515e+00   2.04020e-02   4.76754e-05
   2  p1           8.79142e+02   6.89720e-01   1.19156e-02  -7.73695e-05
   3  p2           8.82595e+00   5.41234e-01   7.87004e-03  -4.16183e-05
   4  p3           7.26699e+00   8.62581e-02   1.47449e-03   2.21378e-03
